<a href="https://colab.research.google.com/github/MaximTislenko/MollekBiolog/blob/main/Zad_02_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Введение
Для изучения выбрана достаточно редкое заболеваие - Фавизм

##Основные положения
* Дефицит глюкоза-6-фосфат-дегидрогеназы (Г6ФД) является наиболее распространенным наследственным нарушением метаболизма красных клеток крови и, при наличии провоцирующих факторов, может вызывать гемолиз.
* Заболеваемость выше в определенных этнических группах (например, люди африканского, средиземноморского или азиатского происхождения).
* Триггеры включают в себя острые заболевания (например, инфекции), лекарственные средства (например, салицилаты) и другие вещества (например, кормовые бобы), которые вызывают окислительный стресс.
* Диагностика проводится с использованием периферического мазка и анализа Г6ФД, ложно-отрицательные результаты анализа на Г6ФД возможны во время острого гемолиза, поэтому, если первоначальный анализ на Г6ФД отрицательный, следует повторить тестирование через несколько недель.
* Избегайте триггеров, чтобы ограничить гемолитические эпизоды.

###В своей практике, я несколько раз встречался с этим заболеванием. Впервые во время ординатуры, тяжелый случай с внутрисосудистым гемолизом. Второй раз - ребенок с тяжелой пневмонией, которому было необходимо было подобрать безопасный антибиотик В третий раз я уже подтвердил дефицит лабораторно, проводя дифференциальную диагностику анемии у подростка.

##При более глубоком изучении проблемы выяснил, что за данное заболевание отвечают два гена, описание ниже.

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.9 MB/s eta 0:00:00


In [ ]:
from Bio import Entrez
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import xml.etree.ElementTree as ET

##Два гена:
* id_01=8573
* id_02=2539

Хочется сделать функцию, которая по $id$ $gens$ сразу выдает цепчку 

In [ ]:
def dna_01(X):
  Entrez.email = "maximtarkovski@gmail.com"
  handle = Entrez.efetch(db="gene", id=X, rettype = 'xml')
  gene_locus = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
  region = gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
  start = int(region.find("Seq-interval_from").text) + 1
  end = int(region.find("Seq-interval_to").text) + 1
  gi_id = region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
  strand = region.find("Seq-interval_strand/Na-strand").get("value")
  # chrom = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
  сhromosome = gene_locus.find("Gene-commentary/Gene-commentary_accession").text + '.' + gene_locus.find("Gene-commentary/Gene-commentary_version").text
  handle_01 = Entrez.efetch(db="nucleotide", id= сhromosome, rettype = 'fasta', retmode = 'text')
  chromosome_loaded = SeqIO.read(handle_01, 'fasta')
  gene_seq = Seq(chromosome_loaded.seq[start:end])
  outputname = '/content/gen_' + str(X) + '.fasta'
  gene_record = SeqIO.SeqRecord(gene_seq, id= str(X), name='no_name', description='Study_05_02_2023')
  SeqIO.write(gene_record, outputname, 'fasta')
  print('Длина_гена - ',len(gene_seq))

In [ ]:
dna_01(8573)

Длина_гена -  408620


In [ ]:
dna_01(2539)

Длина_гена -  16179


In [ ]:
read = SeqIO.read('/content/gen_2539.fasta', 'fasta')
read.seq
print(read.seq)
# print(type(G6PD_2539))

GGCTGTTTGCGGATTTAATGGCAGGGCATTGAGGTTGGGAGGGGTCCCAGCTGCTGCGTCTGCTTTTCTTATAGCAGAGAGGCTGCCTACGGGTGGCACGGGGTGGCCATGGAGTGCAGAGTTGGTGGGACAGGGGACATCCAGGGGGCTCGAGATGTTGCTGGTGACAAGGAATGTCAAGTGGCACTGAGGCTGGCGGGCAAGGCCACAGGCAGATTCTCTCACGTGGGTGCTCGCCCCTTTCCTCCCCCTCGTCCCTCCCTCCCACCCTGGCCCCACTCAGGAGTGAGACCCAGTGGCCAATAAGCTCTGGGACAGACGAATGGGCGCCCTCCTCCTTCCTTCTGTTGGGCTGGAGTGAGTGGAGGAGGTGACTCAGCTCCTGGGCTCAGGCAGGGTCTGGAGGGGCCAGGATGGTCTCGAGTGCTTGGCAGCTGAGGAATGTAGCTGGGCTCGGGTAGTAGCAGCAGCGAGGGGCGGGCCAGGGTGGCCAGAGCCCGGGGCCAGGAATGTGCAGCTGAGGTCAATGGTCCCGGAGTCCTCCCGACTCGGGGTCGGGCGGCGGGAAGGAGGGTGGCCGTGGCGGGGGTGGAGGTGGGTGCCCAGGGCTCAGAGCTTGTGGGGGTTCACCCACTTGTAGGTGCCCTCATACTGGAAACCCACTCTCTTCATCAGCTCGTCTGCCTCCGTGGGGCCTCGGCTGGAGAGTGACGGGTGGAGGAGAGGCATGAGGTAGCTCCACCCTCACCCCGCCCCTGCCCGCTGGGCTCTGTCCCCAGCCCCCACCCTTTCCTCACCTGCCATAAATATAGGGGATGGGCTTGGGCTTCTCCAGCTCAATCTGGTGCAGCAGTGGGGTGAAAATACGCCAGGCCTCACGGAGCTCGTCGCTGAGGGGACATGGTATGGCTTGGGAGGCCGGTGGCACACAGGGAGGGAGGGCAAAGGCCACCCCATAGCCCACAGGTATGCAGGGGCCGGCAGCTGGGCCTCACCTGCG

In [ ]:
rna_G6PD_2539 = read.seq.reverse_complement().transcribe()
print(rna_G6PD_2539)
print(len(rna_G6PD_2539))

GAACUCGGGAAGCCGGCGAGAAGUGUGAGGCCGCGGUAGGGCCGCAUCCCGCUCCGGAGAGAAGUCUGAGUCCGCCAGGCUCUGCAGGCCCGCGGAAGCUCGGUAAUGAUAAGCACGCCGGCCACUUUGCAGGGCGUCACCGCCUACACGCCCCCUCGUCUCUCGGACGGCGGCGUCUAGCCUCGGGGCGCUCGGCCGCCCCGCCCUCUCCGGGGGAGGAAUCAAGAAGAGACUGCCCAAUAGGGCCGGCUUGACCCGCGAACAGGCGAGGGUUCCCGGGGGAGUGGCGCGGCAGAAGGCCCCGCCCAGGAGCCGAGGGACAGCCCAGAGGAGGCGUGGCCACGCUGCCGGCGGAAGUGGAGCCCUCCGCGAGCGCGCGAGGCCGCCGGGGCAGGCGGGGAAACCGGACAGUAGGGGCGGGGCCGGGCCGGCGAUGGGGAUGCGGGAGCACUACGCGGAGCUGCACCCGUGCCCGCCGGAAUUGGGGAUGCAGAGCAGCGGCAGCGGGUAUGGCAGGCAGCCGGCGGGCCGGCCUCCAGCGCAGGUGCCCGAGAGGCAGGGGCUGGCCUGGGAUGCGCGCGCACCUGCCCUCGCCCCGCCCCGCCCGCACGAGGGGUGGUGGCCGAGGCCCCGCCCCGCACGCCUCGCCUGAGGCGGGUCCGCUCAGCCCAGGCGCCCGCCCCCGCCCCCGCCGAUUAAAUGGGCCGGCGGGGCUCAGCCCCCGGAAACGGUCGUACACUUCGGGGCUGCGAGCGCGGAGGGCGACGACGACGAAGCGCAGGUAACCGGCCGGGCGGGCGCCGCGCAGGCGGAGGAGCGUACUGUCCCGCGCUGCGCCGCGCGGCGGUAAAAUACACGCUGUUUGUUGUGCUUGAGAACCGAGCAGAAUCGAGAGGGUCUUAACCAAUCCCUUUAUACCCCGCACCUCCUCUCUUGAGCCCCUGAGACCCCGAGAGCGAAGGGGACUUGCCGACCGGGGUCACCCAGCUUGGCAAGGGGA

In [ ]:
protein_G6PD = rna_G6PD_2539.translate(table= "Vertebrate Mitochondrial", stop_symbol= ' ')
print(protein_G6PD)
print(len(protein_G6PD))

ELGKPA SV PR GRIPLR EVWVRQALQARGSSVMMSTPATLQGVTAYTPPRLSDGGV PRGARPPRPLRG NQEETAQ GRLDPRTGEGSRGSGAAEGPAQEP DSPEEAWPRCRRKWSPPRAREAAGAGGETGQ GRGRAGDGDAGALRGAAPVPAGIGDAEQRQRVWQAAGGPASSAGAREAGAGLGCARTCPRPAPPARGVVAEAPPRTPRL RVRSAQAPAPAPAD MGRRGSAPGNGRTLRGCERGGRRRRSAGNRPGGRRAGGGAYCPALRRAAVKYTLFVVLENRAES GS PIPLYPAPPLLSPWDPESEGDLPTGVTQLGKG AGAELQHLHHLPCS SLWSSRYSREWDGPGHAVPCPT ILLGCPELML SEYTNALSNFVMYTLNEGCLGESGWSPEPAVSNPSTTPQCPAVHKEWFGQSGVTLVWDP GTLKKGANFSMLSCSSALLTSLSST QRHG AGGPEPDPGVRDPAG AFPGRCLPSVGYTHIHHHGCIGEYLPGPNLKSQEVPAPCLSFSN LLQGPTGSAKLSSSSRSC CA IIPALPGVHMHLQDFFFFFFLETEFCSGRPGWGAVAPSWLTATSASWVQAILLPQPPK L LQACTTTPG FLYC   RSFTMLT LVSNSWPQVIRPPRPPKVLGWQAWATTPGC TFLNPVPSRWM PLILELQAL GLS WTSWGKQGMLFIL FFQTCSCLNKWVPKAGSEAWSLPLH HFKFRRYKHLAFLWTLSLCLNFESLP SSASPHLAPIVQLVEPPFCTH KDK MV HEDFHG GG GGP TAIARD STLPCVPWGSETVQGLVEQGLHLKLHVVSVFTFNPGCTQESLCSLLYDHLCH LTL PWAGALSKLVSSSTSCAPQCWLSSVVLCSH TTRCHTLSNLLLDTFVLLI T K  AK EWGGSRG  ALPVAE GGAIVLWLPMSL P  GT WSPQ IGLGLSPSPSLVPSEEPYLAGHGGSRL

In [ ]:
protein_G6PD_list = str(protein_G6PD).split()
protein_G6PD_list
# print((max(protein_G6PD_list, key=len))

['ELGKPA',
 'SV',
 'PR',
 'GRIPLR',
 'EVWVRQALQARGSSVMMSTPATLQGVTAYTPPRLSDGGV',
 'PRGARPPRPLRG',
 'NQEETAQ',
 'GRLDPRTGEGSRGSGAAEGPAQEP',
 'DSPEEAWPRCRRKWSPPRAREAAGAGGETGQ',
 'GRGRAGDGDAGALRGAAPVPAGIGDAEQRQRVWQAAGGPASSAGAREAGAGLGCARTCPRPAPPARGVVAEAPPRTPRL',
 'RVRSAQAPAPAPAD',
 'MGRRGSAPGNGRTLRGCERGGRRRRSAGNRPGGRRAGGGAYCPALRRAAVKYTLFVVLENRAES',
 'GS',
 'PIPLYPAPPLLSPWDPESEGDLPTGVTQLGKG',
 'AGAELQHLHHLPCS',
 'SLWSSRYSREWDGPGHAVPCPT',
 'ILLGCPELML',
 'SEYTNALSNFVMYTLNEGCLGESGWSPEPAVSNPSTTPQCPAVHKEWFGQSGVTLVWDP',
 'GTLKKGANFSMLSCSSALLTSLSST',
 'QRHG',
 'AGGPEPDPGVRDPAG',
 'AFPGRCLPSVGYTHIHHHGCIGEYLPGPNLKSQEVPAPCLSFSN',
 'LLQGPTGSAKLSSSSRSC',
 'CA',
 'IIPALPGVHMHLQDFFFFFFLETEFCSGRPGWGAVAPSWLTATSASWVQAILLPQPPK',
 'L',
 'LQACTTTPG',
 'FLYC',
 'RSFTMLT',
 'LVSNSWPQVIRPPRPPKVLGWQAWATTPGC',
 'TFLNPVPSRWM',
 'PLILELQAL',
 'GLS',
 'WTSWGKQGMLFIL',
 'FFQTCSCLNKWVPKAGSEAWSLPLH',
 'HFKFRRYKHLAFLWTLSLCLNFESLP',
 'SSASPHLAPIVQLVEPPFCTH',
 'KDK',
 'MV',
 'HEDFHG',
 'GG',
 'GGP',
 'TAIARD',
 'STLPCVPWGSE

In [ ]:
print(len(protein_G6PD_list))
print(max(protein_G6PD_list, key=len))
print(len(max(protein_G6PD_list, key=len)))

270
KFLGTHSQPICPAPGNRWSDFCHNGLHLWCWDFPVLLSSIRCMNAQFAGSHVNSSVWLVSSLGPLWTKAAVTIRVQVFVWTCTFISLGVERPGH
94
